In [1]:
!pip install bokeh


# Monitor System Bottlenecks and Profile Framework Operators using Amazon Debugger

This notebook provides an introduction to interactive analysis of the data captured by SageMaker Debugger.

## Table of Contents

* [1. Install and import the latest SageMaker Python SDK](#1)<br>
    * [1.1. Import Debugger classes for deep profiling](#1-1)<br>
    * [1.2. Configure the profiler configuration parameter](#1-2)<br>
* [2. Access profiled data for analysis](#2)<br>
* [3. Analysis by training job phase](#analysis)<br>
    * [3.1. Initialization](#initialization)<br>
    * [3.2. Training loop](#training)<br>
    * [3.3. Finalization](#finalization)<br>
* [Conclusion](#conclusion)

<a class="anchor" id="1"></a>

## 1. Import SageMaker and configure Debugger for deep profiling

To use the new Debugger profiling features, ensure that you have the latest versions of SageMaker (2.19.0 or later) and SMDebug libraries installed.

<a class="anchor" id="1-1"></a>

### 1.1. Import SageMaker and Debugger classes for deep profiling

In this notebook example, we will run a TensorFlow training job to demonstrate how to use the Debugger profiling feature and the SMDebug analysis tools. To get started, let's import the SageMaker TensorFlow estimator class and the new Debugger classes for profiling configuration.

In [2]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import (
    ProfilerConfig,
    FrameworkProfile,
    DetailedProfilingConfig,
    DataloaderProfilingConfig,
    PythonProfilingConfig,
)

Check the current SageMaker IAM role and AWS Region.

In [3]:
import boto3

role = sagemaker.get_execution_role()
print("RoleArn:", role)

session = boto3.session.Session()
region = session.region_name
print("Region:", region)

RoleArn: arn:aws:iam::688520471316:role/hongshan-sagemaker-experiment
Region: us-west-2


<a class="anchor" id="1-2"></a>
### 1.2. Configure the `profiler_config` parameter

In the following code cell, we configure the framework profiling options using the `ProfilerConfig` class. There are two parameters to pass to the `ProfilerConfig` class: `system_monitor_interval_millis` and `framework_profile_params`. The `system_monitor_interval_millis` is to adjust saving intervals for system metrics, and the `framework_profile_params` parameter is to configure how to save framework metrics. For more information about Debugger profiling configurations, see [Configure Debugger Using Amazon SageMaker Python SDK](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-configuration.html) in the [Amazon SageMaker Debugger developer guide](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html).

In [4]:
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(
        local_path="/opt/ml/output/profiler/",
        detailed_profiling_config=DetailedProfilingConfig(start_step=5, num_steps=3),
        dataloader_profiling_config=DataloaderProfilingConfig(start_step=5, num_steps=2),
        python_profiling_config=PythonProfilingConfig(start_step=9, num_steps=1),
    ),
)

In [5]:
estimator = TensorFlow(
    role=sagemaker.get_execution_role(),
    image_uri=f"763104351884.dkr.ecr.{region}.amazonaws.com/tensorflow-training:2.3.1-gpu-py37-cu110-ubuntu18.04",
    instance_count=1,
    instance_type="ml.p3.8xlarge",
    entry_point="train_tf.py",
    profiler_config=profiler_config,
)

In [6]:
estimator.fit(wait=True)

<a class="anchor" id="2"></a>

## 2. Access profiled data for analysis

Once the training job initiates, SageMaker Debugger starts collecting system and framework metrics. The SMDebug library provides tools to access and analyze the profiling data. The following code cells are to set up a `TrainingJob` object to retrieve the system and framework metrics when they become available in the default S3 bucket. Once the metrics are available, you can query, plot, and analyze the profiling metrics data throughout this notebook.

In [7]:
training_job_name = estimator.latest_training_job.job_name

In [16]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)

tj.wait_for_sys_profiling_data_to_be_available()

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-west-2-688520471316/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 5, "NumSteps": 2, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 5, "NumSteps": 3, }', 'FileOpenFailThreshold': '50', 'LocalPath': '/opt/ml/output/profiler/', 'PythonProfilingConfig': '{"StartStep": 9, "NumSteps": 1, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760'}}
s3 path:s3://sagemaker-us-west-2-688520471316/tensorflow-training-2021-06-02-19-13-45-457/profiler-output


Profiler data from system is available


The following SMDebug PandasFrame class converts the profiled data into Pandas frames.

In [17]:
from smdebug.profiler.analysis.utils.profiler_data_to_pandas import PandasFrame

pf = PandasFrame(tj.profiler_s3_output_path)

[2021-06-02 19:26:21.495 ip-172-31-33-148:29665 INFO algorithm_metrics_reader.py:192] S3AlgorithmMetricsReader created with bucket:sagemaker-us-west-2-688520471316 and prefix:tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/


Load the data into the following `system_metrics_df` and `framework_metrics` objects. 

**Note**: This loads the profiled data into memory of your Studio app or Notebook instance. If using the basic instance (ml.t3.medium) and having out of memory issue throughout the notebook,, consider to increase the instance size.

In [18]:
system_metrics_df = pf.get_all_system_metrics()
framework_metrics_df = pf.get_all_framework_metrics(
    selected_framework_metrics=["Step:ModeKeys.TRAIN", "Step:ModeKeys.GLOBAL"]
)

[2021-06-02 19:26:24.512 ip-172-31-33-148:29665 INFO metrics_reader_base.py:134] Getting 9 event files
[2021-06-02 19:26:26.712 ip-172-31-33-148:29665 INFO metrics_reader_base.py:134] Getting 6 event files
[2021-06-02 19:26:26.924 ip-172-31-33-148:29665 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1622661602115635
[2021-06-02 19:26:26.925 ip-172-31-33-148:29665 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1622661599784387
[2021-06-02 19:26:27.032 ip-172-31-33-148:29665 ERROR tf_profiler_parser.py:43] Can't open TF trace file s3://sagemaker-us-west-2-688520471316/tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/tensorflow/detailed_profiling/2021060219/000000005/plugins/profile/2021_06_02_19_23_02/ip-10-0-174-181.us-west-2.compute.internal.trace.json.gz: Exception Error -5 while decompressing data: incomplete or truncated stream 
Reading event from tracefile:s3://sagemaker-us-west-2-688520471316/tensorflow-train

<a id='analysis'></a>

## 3. Analysis profiled data at different training phases

Analysis on the profiled data is framework-specific because the operation IDs, names, and annotations are different across the deep learning frameworks. In this notebook example, we focus on TensorFlow framework profiling.

Import utility classes for visualization and analysis from the SMDebug library.

In [20]:
import os
from IPython.display import IFrame, display, Markdown
from smdebug.profiler.python_profile_utils import StepPhase, PythonProfileModes
from smdebug.profiler.analysis.utils.python_profile_analysis_utils import Metrics
from smdebug.profiler.analysis.python_profile_analysis import PyinstrumentAnalysis, cProfileAnalysis
from smdebug.profiler.analysis.utils.pandas_data_analysis import (
    PandasFrameAnalysis,
    StatsBy,
    Resource,
)

#### Create a Pandas frame analysis object for the profiled data

In [21]:
pf_analysis = PandasFrameAnalysis(system_metrics_df, framework_metrics_df)

#### Identifying the time interval of each phase

The following `get_job_statistics()` function retrieves training start and end time, duration of the initialization, training, and finalization stages.

In [22]:
job_statistics = pf_analysis.get_job_statistics()

initialization_start = job_statistics["start_time"]
initialization_end = job_statistics["training_loop_start"]

training_start = job_statistics["training_loop_start"]
training_end = job_statistics["training_loop_end"]

finalization_start = job_statistics["training_loop_end"]
finalization_end = job_statistics["end_time"]

In [23]:
job_statistics

                                                 0
start_time              2021-06-02T19:16:33:974962
end_time                2021-06-02T19:24:58:975346
job_duration                             505000384
training_loop_start     2021-06-02T19:21:31:278604
training_loop_end       2021-06-02T19:24:12:796763
training_loop_duration                   161518159
initialization                           331278604
finalization                              46178583
initialization_%                           65.5997
training_loop_%                            31.9838
finalization_%                             9.14427

<a id='initialization'></a>

### 3.1 Initialization

This phase of the training job marks the time interval from the start of the training job in SageMaker to the start of the training loop.

#### Python profiling

Let's look at the python profiling stats for functions that were called during initialization.

#### Download Python stats data

Retrieve the Debugger profiling configuration of the training job.

In [24]:
tj.profiler_config["ProfilingParameters"]

{'DataloaderProfilingConfig': '{"StartStep": 5, "NumSteps": 2, "MetricsRegex": ".*", }',
 'DetailedProfilingConfig': '{"StartStep": 5, "NumSteps": 3, }',
 'FileOpenFailThreshold': '50',
 'LocalPath': '/opt/ml/output/profiler/',
 'PythonProfilingConfig': '{"StartStep": 9, "NumSteps": 1, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }',
 'RotateFileCloseIntervalInSeconds': '60',
 'RotateMaxFileSizeInBytes': '10485760'}

Pick up the Python profiling configuration and create an object that takes the configuration JSON strings.

In [25]:
python_stats_dir = "python_stats"
os.makedirs(python_stats_dir, exist_ok=True)
python_profiling_config_tj = eval(
    tj.profiler_config["ProfilingParameters"].get("PythonProfilingConfig")
)

In [26]:
python_profiling_config_tj

{'StartStep': 9,
 'NumSteps': 1,
 'ProfilerName': 'cprofile',
 'cProfileTimer': 'total_time'}

Set an object that automatically detects which Python profiler is used for the training job.

In [27]:
use_pyinstrument = (
    False if python_profiling_config_tj.get("ProfilerName") == "cprofile" else True
)  # Set to True if you used Pyinstrument for Python profiling option.

If you profiled with `cProfile`, you will see the function stats for the top 10 functions with greatest cumulative time spent. If you prefer to use a different metric or a different `n`, modify the `display_python_profile_stats` function defined in the following cell. If you profiled with `Pyinstrument`, you will see the stats separated by each phase of the step.

In [28]:
def display_python_profile_stats(stats):
    if use_pyinstrument:
        if stats:
            for step_stats in stats:
                display(IFrame(src=step_stats.html_file_path, width=1000, height=500))
    else:
        if stats:
            stats.print_top_n_functions(Metrics.CUMULATIVE_TIME, n=10)

The `cProfileAnalysis` and `PyinstrumentAnalysis` classes have functionality to download the Python stats data from the S3 bucket to a specified local directory. The following cell saves the Python stats data into `python_stats` folder under your current working directory, and lists the head of the Python stats data.

In [29]:
python_analysis_class = PyinstrumentAnalysis if use_pyinstrument else cProfileAnalysis
python_analysis = python_analysis_class(
    local_profile_dir=python_stats_dir, s3_path=tj.profiler_s3_output_path
)
step_stats_df = python_analysis.list_profile_stats()
step_stats_df.head()

[2021-06-02 19:27:06.342 ip-172-31-33-148:29665 INFO python_profile_analysis.py:63] Refreshing python profile stats.
[2021-06-02 19:27:07.606 ip-172-31-33-148:29665 INFO python_stats_reader.py:88] Unknown file tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/pevents/2021060219/1622661779048045_81-algo-1_pythontimeline.json found, skipping...
[2021-06-02 19:27:07.606 ip-172-31-33-148:29665 INFO python_stats_reader.py:88] Unknown file tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/pevents/2021060219/1622661781501111_81-algo-1_model_timeline.json found, skipping...
[2021-06-02 19:27:07.607 ip-172-31-33-148:29665 INFO python_stats_reader.py:88] Unknown file tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/pevents/2021060219/1622661781581046_81-algo-1_model_timeline.json found, skipping...
[2021-06-02 19:27:07.607 ip-172-31-33-148:29665 INFO python_stats_reader.py:88] Unknown file tensorflow-training-2021-06-02-19-13-45-457/pr

,profiler_name,framework,node_id,start_mode,start_step,start_phase,start_time_since_epoch_in_micros,end_mode,end_step,end_phase,end_time_since_epoch_in_micros,stats_path
0,cprofile,tensorflow,81-algo-1,prestepzero,-1,StepPhase.START,1.622662e+15,train,0.0,StepPhase.STEP_START,1.622662e+15,python_stats/python_stats/81-algo-1/prestepzer...
1,cprofile,tensorflow,81-algo-1,train,9,StepPhase.STEP_START,1.622662e+15,train,9.0,StepPhase.STEP_END,1.622662e+15,python_stats/python_stats/81-algo-1/train-9-st...
2,cprofile,tensorflow,81-algo-1,train,9,StepPhase.STEP_END,1.622662e+15,train,10.0,StepPhase.STEP_START,1.622662e+15,python_stats/python_stats/81-algo-1/train-9-st...
3,cprofile,tensorflow,81-algo-1,eval,9,StepPhase.STEP_START,1.622662e+15,eval,9.0,StepPhase.STEP_END,1.622662e+15,python_stats/python_stats/81-algo-1/eval-9-ste...
4,cprofile,tensorflow,81-algo-1,eval,9,StepPhase.STEP_END,1.622662e+15,eval,10.0,StepPhase.STEP_START,1.622662e+15,python_stats/python_stats/81-algo-1/eval-9-ste...


The following `fetch_pre_step_zero_profile_stats` function of the `python_analysis` object retrieves profiled data from the pre-step (denoted as start_step `-1` in the output data).

In [30]:
stats = python_analysis.fetch_pre_step_zero_profile_stats()
display_python_profile_stats(stats)

[2021-06-02 19:27:08.547 ip-172-31-33-148:29665 INFO python_profile_analysis.py:63] Refreshing python profile stats.
[2021-06-02 19:27:09.360 ip-172-31-33-148:29665 INFO python_stats_reader.py:88] Unknown file tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/pevents/2021060219/1622661779048045_81-algo-1_pythontimeline.json found, skipping...
[2021-06-02 19:27:09.360 ip-172-31-33-148:29665 INFO python_stats_reader.py:88] Unknown file tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/pevents/2021060219/1622661781501111_81-algo-1_model_timeline.json found, skipping...
[2021-06-02 19:27:09.361 ip-172-31-33-148:29665 INFO python_stats_reader.py:88] Unknown file tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/pevents/2021060219/1622661781581046_81-algo-1_model_timeline.json found, skipping...
[2021-06-02 19:27:09.361 ip-172-31-33-148:29665 INFO python_stats_reader.py:88] Unknown file tensorflow-training-2021-06-02-19-13-45-457/pr

<a id='training'></a>

### 3.2 Training loop

This phase of the training job is time spent solely on training. It includes the data loading process, forward and backward pass, and synchronization.

#### Utilization histograms

`MetricHistogram` computes a histogram on GPU and CPU utilization values. Bins are between 0 and 100. Good system utilization means that the center of the distribtuon should be between 80 to 90. 

The following cell will plot the histograms per metric. In order to only plot specific metrics define the list  `select_dimensions` and `select_events`. A dimension can be CPUUtilization, GPUUtilization, GPUMemoryUtilization, and IOPS (IO per second). If no event is specified, CPU uiltization histograms for each single core and total cpu usage will be plotted. In case of GPU, it will visualize utilization and memory for each GPU. In case of IOPS, it will plot IO wait time per cpu. If `select_events` is specified, only those metrics that match the name in `select_metrics` will be shown. If neither `select_dimensions` nor `select_events` are specified, all available metrics will be visualized.

In [31]:
from smdebug.profiler.analysis.notebook_utils.metrics_histogram import MetricsHistogram

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

metrics_histogram = MetricsHistogram(system_metrics_reader)
metrics_histogram.plot(
    starttime=0,
    endtime=system_metrics_reader.get_timestamp_of_latest_available_file(),
    select_dimensions=["CPU", "GPU"],
    # select_events=["total"]
)

[2021-06-02 19:27:09.536 ip-172-31-33-148:29665 INFO metrics_reader_base.py:134] Getting 9 event files
Found 71369 system metrics events from timestamp_in_us:0 to timestamp_in_us:1622661840000000
select events:['.*']
select dimensions:['CPU', 'GPU']
filtered_events:{'cpu6', 'cpu2', 'cpu20', 'cpu11', 'cpu17', 'cpu16', 'cpu3', 'cpu5', 'cpu4', 'cpu25', 'IOPS', 'cpu29', 'gpu0', 'cpu14', 'gpu1', 'cpu8', 'cpu22', 'cpu13', 'TransmitBytesPerSecond', 'cpu30', 'cpu27', 'total', 'cpu7', 'cpu1', 'cpu24', 'cpu26', 'ReadThroughputInBytesPerSecond', 'cpu10', 'cpu9', 'MemoryUsedPercent', 'WriteThroughputInBytesPerSecond', 'cpu19', 'ReceiveBytesPerSecond', 'cpu21', 'cpu12', 'cpu31', 'gpu3', 'cpu0', 'cpu28', 'cpu18', 'cpu23', 'gpu2', 'cpu15'}
filtered_dimensions:{'GPUMemoryUtilization-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1'}


filtered_dimensions:{'GPUMemoryUtilization-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1'}


#### Step durations over time

SageMaker Debugger records the durations of each step, which is the time spent in one forward and backward pass. The following code cell plots step durations (y-axis) over training job duration (x-axis). Typically it's expected that the step durations should be similar across the training run. Signficant step duration outliers indicate that there might be bottleneck issues related to those steps. `StepTimelineChart` helps to identify if such outliers happen in regular intervals.

In [32]:
tj.wait_for_framework_profiling_data_to_be_available()

from smdebug.profiler.analysis.notebook_utils.step_timeline_chart import StepTimelineChart

framework_metrics_reader = tj.get_framework_metrics_reader()
framework_metrics_reader.refresh_event_file_list()

view_step_timeline_chart = StepTimelineChart(framework_metrics_reader)

[2021-06-02 19:27:11.470 ip-172-31-33-148:29665 INFO algorithm_metrics_reader.py:192] S3AlgorithmMetricsReader created with bucket:sagemaker-us-west-2-688520471316 and prefix:tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/


 Profiler data from framework is available
Found recorded framework annotations. Latest available timestamp microsseconds_since_epoch is:1622661853613895 , human_readable_timestamp in utc: 2021-06-02T19:24:13:613895
[2021-06-02 19:27:11.642 ip-172-31-33-148:29665 INFO metrics_reader_base.py:134] Getting 7 event files
[2021-06-02 19:27:11.846 ip-172-31-33-148:29665 INFO algorithm_metrics_reader.py:233] Invalid tracefilename:s3://sagemaker-us-west-2-688520471316/tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/tensorflow/detailed_profiling/2021060219/000000005/plugins/profile/2021_06_02_19_23_02/ip-10-0-174-181.us-west-2.compute.internal.memory_profile.json.gz . Skipping.
[2021-06-02 19:27:11.847 ip-172-31-33-148:29665 INFO

#### Python profiling

Let's now look at the python profiling stats for functions that were called during the first step profiled. From the Python profiling configuration JSON of your training job, call the start step of profiled data from the training loop.

In [33]:
python_profiling_config_tj["StartStep"]

9

Display a list of Python operators executed during the profiled step.

In [34]:
if python_profiling_config_tj == {}:
    step = 9
else:
    step = int(python_profiling_config_tj["StartStep"])
phase = StepPhase.STEP_START
mode = PythonProfileModes.TRAIN
python_step_stats = python_analysis.fetch_profile_stats_by_step(
    start_step=step, end_step=step + 1, start_phase=phase, end_phase=phase, mode=mode
)
display_python_profile_stats(python_step_stats)

Wed Jun  2 19:27:13 2021    python_stats/python_stats/81-algo-1/train-9-stepstart-1622661783621442.2_train-9-stepend-1622661783726648.2/python_stats
Wed Jun  2 19:27:13 2021    python_stats/python_stats/81-algo-1/train-9-stepend-1622661783731965.8_train-10-stepstart-1622661783737516.5/python_stats

         3853 function calls (3708 primitive calls) in 0.110 seconds

   Ordered by: cumulative time
   List reduced from 292 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.070    0.070 /usr/local/lib/python3.7/site-packages/tensorflow/python/eager/function.py:2827(__call__)
        1    0.000    0.000    0.070    0.070 /usr/local/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py:757(__call__)
        1    0.000    0.000    0.070    0.070 /usr/local/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py:798(_call)
        1    0.060    0.060    0.060    0.060 {built-in meth

If detailed profiling was enabled and cProfile was used as the Python profiler, we can also just look at the stats for the training loop as a whole.

In [35]:
if use_pyinstrument:
    print("Pyinstrument was used as the Python profiler, no training loop stats are available.")
else:
    python_job_stats = python_analysis.fetch_profile_stats_by_job_phase()
    training_loop_stats = python_job_stats["training_loop"]
    display_python_profile_stats(training_loop_stats)

No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested interval!
No stats were found for the requested in

#### CPU/GPU utilization

SageMaker Debugger also records system metrics (resource utilization) for the training job. The following timeline charts depict the utilization per core and GPU. It shows the last 1000 datapoints. You can inspect previous datapoints by zooming out of the chart.

In [36]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
framework_metrics_reader.refresh_event_file_list()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

[2021-06-02 19:27:14.212 ip-172-31-33-148:29665 INFO metrics_reader_base.py:134] Getting 9 event files
select events:['total']
select dimensions:['CPU', 'GPU']
filtered_events:{'total'}
filtered_dimensions:{'GPUMemoryUtilization-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1'}


#### Drill down by choosing a target range, get the exact time range you chose, and output correlated framework metrics

To drill down into an interested time range you want to investigate further, you can choose a range on the preceeding graphs and get the exact time range. As shown in the following animated screenshot, copy the time range that you chose and paste to the following two cells. 
<IMG src=images/select_range_drilldown_framework_metrics.gif/>

1. Find time annotation of every framework operation.

In [37]:
# Note change index range below with selected index range from above cell
view_timeline_charts.find_time_annotations([808, 831])

Selected timerange: 1622661798.4752445 to 1622661809.9751134
[2021-06-02 19:27:15.551 ip-172-31-33-148:29665 INFO metrics_reader_base.py:134] Getting 6 event files
Reading event from tracefile:s3://sagemaker-us-west-2-688520471316/tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/tensorflow/detailed_profiling/2021060219/000000005/plugins/profile/2021_06_02_19_23_02/ip-10-0-174-181.us-west-2.compute.internal.trace.json.gz and node_id:ip-10-0-174-181
Spent 0.006328595 ms (cumulative time) in Step:ModeKeys.EVAL_0
Spent 1.4633e-05 ms (cumulative time) in Step:ModeKeys.EVAL_1
Spent 1.5879e-05 ms (cumulative time) in Step:ModeKeys.EVAL_2
Spent 1.3075e-05 ms (cumulative time) in Step:ModeKeys.EVAL_3
Spent 1.2487e-05 ms (cumulative time) in Step:ModeKeys.EVAL_4
Spent 1.3029e-05 ms (cumulative time) in Step:ModeKeys.EVAL_5
Spent 1.2794e-05 ms (cumulative time) in Step:ModeKeys.EVAL_6
Spent 1.2765e-05 ms (cumulative time) in Step:ModeKeys.EVAL_7
Spent 1.2872e-05 ms (cumulativ

2. Plot timeline chart of the framework metrics for the selected time range. To avoid issues with out of memory, it will only plot the first 1000 datapoints.

In [38]:
# Note change index range below with selected index range from above cell
view_timeline_charts.plot_detailed_profiler_data([808, 831])

Selected timerange: 1622661798.475245 to 1622661809.975114
[2021-06-02 19:27:15.821 ip-172-31-33-148:29665 INFO metrics_reader_base.py:134] Getting 6 event files
Reading event from tracefile:s3://sagemaker-us-west-2-688520471316/tensorflow-training-2021-06-02-19-13-45-457/profiler-output/framework/tensorflow/detailed_profiling/2021060219/000000005/plugins/profile/2021_06_02_19_23_02/ip-10-0-174-181.us-west-2.compute.internal.trace.json.gz and node_id:ip-10-0-174-181


#### Identifying CPU bottlenecks

A heatmap is a representation where each row corresponds to one metric (CPU core and GPU utilizations) and x-axis is the duration of the training job. It allows to more easily spot CPU bottlenecks e.g. if utilization on GPU is low but a utilization of one or more cores is high. 

In the below heatmap, Yellow indicates maximum utilization, purple means that utilization was 0. GPUs have frequent stalled cycles where utilization is dropping to 0 while at the same time utilization on CPU cores is at a maximum. This is a clear indication of a CPU bottleneck where GPUs are waiting for the data to arrive. Such a bottleneck can be caused for instance by a compute-heavy pre-processing.

In [39]:
from smdebug.profiler.analysis.notebook_utils.heatmap import Heatmap

system_metrics_reader.refresh_event_file_list()
view_heatmap = Heatmap(system_metrics_reader, plot_height=450)

select events:['.*']
select dimensions:['.*CPU', '.*GPU']
filtered_events:{'cpu6', 'cpu2', 'cpu20', 'cpu11', 'cpu17', 'cpu16', 'cpu3', 'cpu5', 'cpu4', 'cpu25', 'IOPS', 'cpu29', 'gpu0', 'cpu14', 'gpu1', 'cpu8', 'cpu22', 'cpu13', 'TransmitBytesPerSecond', 'cpu30', 'cpu27', 'total', 'cpu7', 'cpu1', 'cpu24', 'cpu26', 'ReadThroughputInBytesPerSecond', 'cpu10', 'cpu9', 'MemoryUsedPercent', 'WriteThroughputInBytesPerSecond', 'cpu19', 'ReceiveBytesPerSecond', 'cpu21', 'cpu12', 'cpu31', 'gpu3', 'cpu0', 'cpu28', 'cpu18', 'cpu23', 'gpu2', 'cpu15'}
filtered_dimensions:{'GPUUtilization', 'CPUUtilization', 'GPUMemoryUtilization'}


#### Merging all the trace events 

Debugger captures events from different sources, such as framework, debugger hook, Horovod (if applicable). Putting them together, aligning the events and visualizing, will give an idea of events occurring in different components of the training job, and identify potential issues. The merge_timeline() API below combines the different types of trace event files into one trace event file, which can then be viewed in the browser.

Below, we merge the events that occurred between step 1 and 10. You can modifying this step interval, or alternatively, merge events within a time interval.

In [40]:
from smdebug.profiler.analysis.utils.merge_timelines import MergedTimeline, MergeUnit

start_step, end_step = 1, 10
combined_timeline = MergedTimeline(tj.profiler_s3_output_path, output_directory="./")
combined_timeline.merge_timeline(start_step, end_step, unit=MergeUnit.STEP)

[2021-06-02 19:27:17.966 ip-172-31-33-148:29665 INFO metrics_reader_base.py:134] Getting 3 event files
[2021-06-02 19:27:18.038 ip-172-31-33-148:29665 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1622661602115635


AttributeError: module 'horovod.torch' has no attribute 'size'

As shown in the following animated screenshot, do the followings:

1. Browse the current working directory to find the combined timeline file.
2. Open a Chrome browser tracing app at [chrome://tracing](chrome://tracing), and load the JSON file.
3. Use the mouse motion tool box to shift or zoom in and out the timeline. You can also use `W`, `A`, `S`, and `D` keys to browse the timeline.

<IMG src=images/merged_timeline.gif>

Next, since data loading is one of the most compute-intensive parts of the training process, we dive deep into analysing profiler data related to data loading.

#### Profiling TensorFlow data loader processes<a id='dataloading'></a>

List the top 10 `Step` metrics from the profiled data. From the output of the following cell, you can find out the time spent on each step and operations executed at each step in detail.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()
system_metrics_reader = tj.get_systems_metrics_reader()

tj.wait_for_framework_profiling_data_to_be_available()
framework_metrics_reader = tj.get_framework_metrics_reader()
framework_metrics_reader.refresh_event_file_list()

from smdebug.profiler.analysis.utils.profiler_data_to_pandas import PandasFrame

pf = PandasFrame(tj.profiler_s3_output_path)
system_metrics_df = pf.get_all_system_metrics()

step_metrics_df = pf.get_all_framework_metrics(selected_framework_metrics=["Step:ModeKeys.TRAIN"])

step_metrics_df.head(10)

List the top 10 data loader process operators. We filter data loading events of using the `GeneratorDatasetOp::Dataset::Iterator::GetNext` regex from a given train step. Measuring the time span of these events are useful to evaluate how the data loading processes are efficient.

In [ ]:
iterator_metrics = pf.get_all_framework_metrics(
    selected_framework_metrics=["tensorflow::data::GeneratorDatasetOp::Dataset::Iterator::GetNext"]
)
iterator_metrics.head(10)

The following cell prints the time taken by first 10 time steps to perform data loading. It also lists the number of workers processed for the data loading.

In [ ]:
for i in range(10):
    cur_step = step_metrics_df.iloc[i]
    step_pid = cur_step["pid"]
    step_tid = cur_step["tid"]
    cur_step_number = cur_step["step"]
    step_start_time = cur_step["start_time_us"]
    step_end_time = cur_step["end_time_us"]
    step_duration = step_end_time - step_start_time

    data_iter_cur_step = iterator_metrics.loc[
        (iterator_metrics["start_time_us"] >= step_start_time)
        & (iterator_metrics["end_time_us"] <= step_end_time)
    ]
    num_workers = data_iter_cur_step.shape[0]
    dl_time_mean = (
        data_iter_cur_step["end_time_us"].mean() - data_iter_cur_step["start_time_us"].mean()
    )
    print(
        f"Step number: {cur_step_number}, time taken by train step: {step_duration}, Number of dataloading workers: {num_workers}, Average dataloading time of all the workers: {dl_time_mean} microseconds"
    )

<a id='finalization'></a>

### 3.3 Finalization

The finalization phase of the training job marks the time interval from the end of the training loop to the end of the training job in SageMaker. The following cell returns the python profiling stats for functions that were called during finalization.

In [ ]:
stats = python_analysis.fetch_post_hook_close_profile_stats()
display_python_profile_stats(stats)

<a id='conclusion'></a>

## Conclusion

In this notebook, we explored the full functionality of the new Debugger profiling features. SageMaker Debugger initiates training job with your preferred profiling settings. The SMDebug client library provides the analysis tools that you can drill down to the profiled data. You can profile each training job phases, watch which steps make bottlenecks and slowdowns at different training phases, and retrieve a list of operations at the target step or time range. To deep dive and learn more about SageMaker Debugger, see the [Amazon SageMaker Debugger developer guide](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html).